In [1]:
import pandas as pd
import numpy as np

## RFM-анализ
## Сегментация пользователей

In [2]:
df = pd.read_csv('~/RFM/RFM_ht_data.csv', low_memory=False)

In [3]:
df.dtypes

InvoiceNo        object
CustomerCode     object
InvoiceDate      object
Amount          float64
dtype: object

In [4]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [5]:
df.columns = df.columns.str.lower()

In [6]:
df['invoicedate'].describe()

/var/folders/st/gl4370793qz1yf8d4zkjm7lr0000gp/T/ipykernel_2793/1515196518.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df['invoicedate'].describe()


count                  332730
unique                     30
top       2020-09-30 00:00:00
freq                    19781
first     2020-09-01 00:00:00
last      2020-09-30 00:00:00
Name: invoicedate, dtype: object

In [7]:
df.shape

(332730, 4)

In [8]:
df.head()

,invoiceno,customercode,invoicedate,amount
0,C0011810010001,19067290,2020-09-01,1716.00
1,C0011810010017,13233933,2020-09-01,1489.74
2,C0011810010020,99057968,2020-09-01,151.47
3,C0011810010021,80007276,2020-09-01,146.72
4,C0011810010024,13164076,2020-09-01,104.00


In [9]:
last_date = df['invoicedate'].max()

1.Какое максимальное кол-во покупок было совершено одним пользователем?  

In [10]:
df.groupby('customercode', as_index=False)\
  .agg({'invoiceno':'nunique'}).sort_values('invoiceno', ascending=False)

,customercode,invoiceno
89388,19057820,204
44594,13215452,113
10347,13032521,106
97077,19080880,99
119951,99003061,90
...,...,...
58910,13272861,1
58911,13272871,1
58913,13272875,1
58914,13272878,1


In [11]:

rfmTable = df.groupby('customercode').agg({'invoicedate': lambda x: (last_date - x.max()).days, # Recency #Количество дней с последнего заказа
                                        'invoiceno': lambda x: len(x),      # Frequency #Количество заказов
                                        'amount': lambda x: x.sum()}) # Monetary Value #Общая сумма по всем заказам

rfmTable['invoicedate'] = rfmTable['invoicedate'].astype(int)
rfmTable.rename(columns={'invoicedate': 'recency', 
                         'invoiceno': 'frequency', 
                         'amount': 'monetary_value'}, inplace=True)

In [12]:
rfmTable.head()

,recency,frequency,monetary_value
customercode,,,
02213019,19,1,1609.20
02213042,22,3,9685.48
02213071,29,1,415.00
02213088,23,1,305.00
02213092,25,1,1412.88


In [13]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])

In [14]:
quantiles

,recency,frequency,monetary_value
0.25,2.0,1.0,765.00
0.50,8.0,2.0,1834.48
0.75,16.0,3.0,4008.84


In [15]:
rfmSegmentation = rfmTable

In [16]:

def RClass(value,parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 1
    elif value <= quantiles_table[parameter_name][0.50]:
        return 2
    elif value <= quantiles_table[parameter_name][0.75]: 
        return 3
    else:
        return 4


def FMClass(value, parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 4
    elif value <= quantiles_table[parameter_name][0.50]:
        return 3
    elif value <= quantiles_table[parameter_name][0.75]: 
        return 2
    else:
        return 1


In [17]:

rfmSegmentation['R_Quartile'] = rfmSegmentation['recency'].apply(RClass, args=('recency',quantiles))

rfmSegmentation['F_Quartile'] = rfmSegmentation['frequency'].apply(FMClass, args=('frequency',quantiles))

rfmSegmentation['M_Quartile'] = rfmSegmentation['monetary_value'].apply(FMClass, args=('monetary_value',quantiles))

rfmSegmentation['RFMClass'] = rfmSegmentation.R_Quartile.map(str)    \
+ rfmSegmentation.F_Quartile.map(str)                       \
+ rfmSegmentation.M_Quartile.map(str)

In [18]:
rfmSegmentation.head()

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
customercode,,,,,,,
02213019,19,1,1609.20,4,4,3,443
02213042,22,3,9685.48,4,2,1,421
02213071,29,1,415.00,4,4,4,444
02213088,23,1,305.00,4,4,4,444
02213092,25,1,1412.88,4,4,3,443


2.Какая верхняя граница у суммы покупок у пользователей с классом 4 в подсегменте М? (Другими словами: пользователи, у которых сумма покупок от 0 до Х попадают в 4 класс в подсегменте М)  

In [19]:
rfmSegmentation.query("M_Quartile==4").monetary_value.max()

765.0

3.Какая нижняя граница у количества покупок у пользователей с классом 1 в подсегменте F?  

In [20]:
rfmSegmentation.query("F_Quartile==1").frequency.min()

4

4.Какое максимальное количество дней может пройти с момента последней покупки для того, чтобы пользователь попал в класс 2 в подсегменте R?

In [21]:
rfmSegmentation.query("R_Quartile==2").recency.max()

8

5.Сколько пользователей попало в сегмент 111? 

In [22]:
rfmSegmentation.query("RFMClass=='111'").shape[0]

9705

6.Сколько пользователей попало в сегмент 311?

In [23]:
rfmSegmentation.query("RFMClass=='311'").shape[0]

1609

7.В каком RFM-сегменте самое большое кол-во пользователей? 

In [24]:
rfmSegmentation.groupby('RFMClass').count().idxmax()

recency           444
frequency         444
monetary_value    444
R_Quartile        444
F_Quartile        444
M_Quartile        444
dtype: object

8.В каком RFM-сегменте самое маленькое кол-во пользователей? 

In [25]:
rfmSegmentation.groupby('RFMClass').count().idxmin()

recency           414
frequency         414
monetary_value    414
R_Quartile        414
F_Quartile        414
M_Quartile        414
dtype: object

9.Какое количество пользователей попало в самый малочисленный сегмент?

In [26]:
rfmSegmentation.query('RFMClass=="414"').shape[0]

2